## Version Check

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.0.11'

## Connecting

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

## Declare a Mapping

In [3]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [4]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s'>" % (self.name, self.fullname, self.password)